In [1]:
import os
import re

In [30]:
# dataset_path = "data/bigbench.csv"
dataset_path = "data/FeynmanEquations.csv"
skip_equations = []
import pandas as pd
df = pd.read_csv(dataset_path).dropna(axis=0, how='all')
print(df.head())
# df = df[~df['Number'].astype(int).isin(skip_equations)]
# bounds = 
# df_str = df["Output"] + " = " + df["Formula"]


  Filename  Number Output                                            Formula  \
0   I.6.2a     1.0      f                        exp(-theta**2/2)/sqrt(2*pi)   
1    I.6.2     2.0      f        exp(-(theta/sigma)**2/2)/(sqrt(2*pi)*sigma)   
2   I.6.2b     3.0      f  exp(-((theta-theta1)/sigma)**2/2)/(sqrt(2*pi)*...   
3   I.8.14     4.0      d                        sqrt((x2-x1)**2+(y2-y1)**2)   
4   I.9.18     5.0      F         G*m1*m2/((x2-x1)**2+(y2-y1)**2+(z2-z1)**2)   

   # variables v1_name  v1_low  v1_high v2_name  v2_low  ...  v7_high v8_name  \
0          1.0   theta     1.0      3.0     NaN     NaN  ...      NaN     NaN   
1          2.0   sigma     1.0      3.0   theta     1.0  ...      NaN     NaN   
2          3.0   sigma     1.0      3.0   theta     1.0  ...      NaN     NaN   
3          4.0      x1     1.0      5.0      x2     1.0  ...      NaN     NaN   
4          9.0      m1     1.0      2.0      m2     1.0  ...      2.0      z1   

   v8_low  v8_high v9_name  v9_l

In [58]:
path1 = "/home/ubuntu/llm-sr/ablation_results_foundry/logs/exp17" # 14
path2 = "/home/ubuntu/llm-sr/ablation_results_foundry/logs/exp15" # 15
path3 = "/home/ubuntu/llm-sr/ablation_results_foundry/logs/exp16" # 16
path4 = "/home/ubuntu/llm-sr/ablation_results_foundry/logs/exp1"
path5 = "/home/ubuntu/llm-sr/ablation_results_foundry/logs/exp_pysr"
path6 = "/home/ubuntu/llm-sr/ablation_results_foundry/logs/exp18"

files = {
    'llama3' : path2,
    # 'llama3-hints' : path1,
    # 'llama3-no-concepts' : path6,
    'pysr' : path5,
}

# GT Equation #96:  m = (h/(2*pi))**2/(2*E_n*d**2)
# Pred Equation #96:  m = (((h * ((h * 0.012665465) / d)) / E_n) / d) / (((sqrt((E_n / h) * d) / 1.1992105) * h) ^ (4.304143e-5 / E_n))
# Score: 1.5665505016670642, Loss: 2.1319321e-08, Complexity: 26

pred_finder = re.compile(r"Pred Equation #(\d+): (.+)")
gt_finder = re.compile(r"GT Equation #(\d+): (.+)")
loss_finder = re.compile(r"Loss: (.+), Complexity: (.+)")
outputs = {}
for name, path in files.items():
    logfile = "/home/ubuntu/llm-sr/ablation_results_foundry/" + os.path.basename(path) + ".log"
    outputs[name] = []
    with open(logfile, "r") as f:
        tup = []
        for line in f:
            pred_match = pred_finder.search(line)
            if pred_match:
                pred_eq = pred_match.group(2)
                tup.append(pred_eq.strip())
            gt_match = gt_finder.search(line)
            if gt_match:
                gt_eq = gt_match.group(2)
                tup.append(gt_eq.strip())
            loss_match = loss_finder.search(line)
            if loss_match:
                loss = loss_match.group(1)
                complexity = loss_match.group(2)
                tup.append(float(loss))
            if len(tup) == 3:
                outputs[name].append(tup)
                tup = []
        # print(len(outputs[name]))
    

In [59]:
by_equation = {}
for name, data in outputs.items():
    for gt, pred, loss in data:
        if gt not in by_equation:
            by_equation[gt] = {}
        by_equation[gt][name] = loss


In [66]:
for k, v in by_equation.items():
    print(k)
    for name, loss in v.items():
        print(f"\t{name}: {loss}")

f = exp(-theta**2/2)/sqrt(2*pi)
	llama3: 0.0004277438
	pysr: 6.172537e-11
f = exp(-(theta/sigma)**2/2)/(sqrt(2*pi)*sigma)
	llama3: 0.3451757
	pysr: 0.0002471752
f = exp(-((theta-theta1)/sigma)**2/2)/(sqrt(2*pi)*sigma)
	llama3: 0.93885386
	pysr: 17.280687
d = sqrt((x2-x1)**2+(y2-y1)**2)
	llama3: 75321.73
	pysr: 22029.773
F = G*m1*m2/((x2-x1)**2+(y2-y1)**2+(z2-z1)**2)
	llama3: 454.66455
	pysr: 402.3858
m = m_0/sqrt(1-v**2/c**2)
	llama3: 32.034485
	pysr: 1.1498109
A = x1*y1+x2*y2+x3*y3
	llama3: 2.0753532e-06
	pysr: 2.1678943e-06
F = mu*Nn
	llama3: 2.941789e-07
	pysr: 2.9629632e-07
K = 1/2*m*(v**2+u**2+w**2)
	llama3: 11245859.0
	pysr: 799088.6
F = q1*q2*r/(4*pi*epsilon*r**3)
	llama3: 4.6709058e-08
	pysr: 2.191505e-06
Ef = q1*r/(4*pi*epsilon*r**3)
	llama3: 5.550575e-08
	pysr: 9.199903e-06
F = q2*Ef
	llama3: 3.0934902e-07
	pysr: 2.9370284e-07
F = q*(Ef+B*v*sin(theta))
	llama3: 7.4534255e-06
	pysr: 6.7511423e-06
U = G*m1*m2*(1/r2-1/r1)
	llama3: 8.4394804e-07
	pysr: 32422.06
U = m*g*z
	llama3: